In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import time
import gc
import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)

## a. Data Pre-Processing

Load data

In [2]:
%%time
historical_transactions = pd.read_csv('../input_data/historical_transactions.csv',
                                      parse_dates=['purchase_date'])
new_merchant_transactions = pd.read_csv('../input_data/new_merchant_transactions.csv',
                                       parse_dates=['purchase_date'])
merchants = pd.read_csv('../input_data/merchants.csv')
train = pd.read_csv('../input_data/train.csv',
                   parse_dates=['first_active_month'])
test = pd.read_csv('../input_data/test.csv',
                   parse_dates=['first_active_month'])

Wall time: 3min 26s


In [3]:
def convert_binary_features(df, cols):
    for col in cols:
        df[col] = df[col].apply(lambda x:1 if x == 'Y' else 0)
        
    return df

*historical_transactions* and *new_merchant_transactions*

In [4]:
%%time

historical_transactions = convert_binary_features(historical_transactions, ['authorized_flag', 'category_1'])
new_merchant_transactions = convert_binary_features(new_merchant_transactions, ['authorized_flag', 'category_1'])

Wall time: 55.5 s


*merchants*

In [5]:
%%time
merchants = convert_binary_features(merchants, ['category_1', 'category_4'])
merchants['category_2'] = merchants['category_2'].fillna(0)

Wall time: 635 ms


*test*

In [6]:
# Missing values
test[test['first_active_month'].isna()]

,first_active_month,card_id,feature_1,feature_2,feature_3
11578,NaT,C_ID_c27b4f80f7,5,2,1


In [7]:
# Impute missing values here

concatenate *train* and *test*

In [8]:
%%time
train['type'] = 0
test['type'] = 1
train_test = pd.concat([train, test], ignore_index=True)

Wall time: 656 ms


In [9]:
%%time
train_test.to_pickle('../processed_data/train_test.pkl')
historical_transactions.to_pickle('../processed_data/historical_transactions.pkl')
new_merchant_transactions.to_pickle('../processed_data/new_merchant_transactions.pkl')
merchants.to_pickle('../processed_data/merchants.pkl')

Wall time: 2min 10s


## b. Feature Engineering

In [10]:
%%time
train_test = pd.read_pickle('../processed_data/train_test.pkl')
historical_transactions = pd.read_pickle('../processed_data/historical_transactions.pkl')
new_merchant_transactions = pd.read_pickle('../processed_data/new_merchant_transactions.pkl')
merchants = pd.read_pickle('../processed_data/merchants.pkl')

Wall time: 29.9 s


### b1. *train_test*

In [11]:
train_test.tail(2)

,card_id,feature_1,feature_2,feature_3,first_active_month,target,type
325538,C_ID_6c46fc5a9d,2,1,0,2017-06-01,NaN,1
325539,C_ID_87e7979a5f,5,1,1,2016-10-01,NaN,1


Label-encode **feature_1** and **feature_2** (**feature_3** is a binary feature so no need to encode)

In [12]:
%%time
train_test = pd.get_dummies(train_test, columns=['feature_1','feature_2'])

Wall time: 206 ms


Convert **first_active_month** to date_time and add an **elapsed_time** feature

In [13]:
%%time
train_test['elapsed_time'] = (datetime.date(2018, 2, 1) - train_test['first_active_month'].dt.date).dt.days

Wall time: 3.13 s


In [14]:
train_test.head(2)

,card_id,feature_3,first_active_month,target,type,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,feature_2_3,elapsed_time
0,C_ID_92a2005557,1,2017-06-01,-0.820283,0,0,0,0,0,1,0,1,0,245.0
1,C_ID_3d0044924f,0,2017-01-01,0.392913,0,0,0,0,1,0,1,0,0,396.0


### b2. *historical_transactions* and *new_merchant_transactions*

In [15]:
historical_transactions.head(2)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,1,C_ID_4e6213e9bc,88,0,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16


Label encode categorical features **category_2**, **category_3**

In [16]:
%%time
historical_transactions = pd.get_dummies(historical_transactions, columns=['category_2','category_3'])
new_merchant_transactions = pd.get_dummies(new_merchant_transactions, columns=['category_2','category_3'])

Wall time: 1min 13s


In [17]:
historical_transactions.head(2)

,authorized_flag,card_id,city_id,category_1,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_3_A,category_3_B,category_3_C
0,1,C_ID_4e6213e9bc,88,0,0,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,16,37,1,0,0,0,0,1,0,0
1,1,C_ID_4e6213e9bc,88,0,0,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,16,16,1,0,0,0,0,1,0,0


Generate datetime related features

In [18]:
%%time
def generate_datetime_features(transactions):
    transactions['purchase_month'] = transactions['purchase_date'].dt.month
    transactions['purchase_year'] = transactions['purchase_date'].dt.year
    transactions['dayofweek'] = transactions['purchase_date'].dt.dayofweek
    transactions['weekend'] = (transactions['purchase_date'].dt.weekday >=5).astype(int)
    return transactions

historical_transactions = generate_datetime_features(historical_transactions)
new_merchant_transactions = generate_datetime_features(new_merchant_transactions)

Wall time: 22.1 s


Generate month diff / month lag. See https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/73244 for more details

In [19]:
%%time
def generate_month_diff(transactions):
    transactions['month_diff'] = ((datetime.datetime.today() - transactions['purchase_date']).dt.days)//30
    transactions['month_diff'] += transactions['month_lag']
    return transactions

historical_transactions = generate_month_diff(historical_transactions)
new_merchant_transactions = generate_month_diff(new_merchant_transactions)

Wall time: 7.07 s


In [20]:
historical_transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_3_A,category_3_B,category_3_C,purchase_month,purchase_year,dayofweek,weekend,month_diff
0,1,C_ID_4e6213e9bc,88,0,0,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,16,37,1,0,0,0,0,1,0,0,6,2017,6,1,16
1,1,C_ID_4e6213e9bc,88,0,0,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,16,16,1,0,0,0,0,1,0,0,7,2017,5,1,16
2,1,C_ID_4e6213e9bc,88,0,0,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,16,37,1,0,0,0,0,1,0,0,8,2017,2,0,17
3,1,C_ID_4e6213e9bc,88,0,0,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,16,34,1,0,0,0,0,1,0,0,9,2017,5,1,17
4,1,C_ID_4e6213e9bc,88,0,0,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,16,37,1,0,0,0,0,1,0,0,3,2017,4,0,17


In [21]:
%%time
def aggregate_transactions(transactions):
    
    agg_func = {
        'authorized_flag' : ['sum', 'mean'],
        'city_id' : ['nunique'],      # mean encoding ?
        'category_1': ['sum', 'mean'],
        'category_2_1.0': ['mean'],
        'category_2_2.0': ['mean'],
        'category_2_3.0': ['mean'],
        'category_2_4.0': ['mean'],
        'category_2_5.0': ['mean'],
        'category_3_A' : ['mean'],
        'category_3_B' : ['mean'],
        'category_3_C' : ['mean'],
        'installments': ['sum', 'mean', 'max', 'min', 'std'],
        'merchant_category_id' : ['nunique'],
        'merchant_id' : ['nunique'], # mean encoding ?
        'purchase_amount' : ['sum', 'mean', 'max', 'min', 'std'],
        'purchase_date': ['min', 'max'],
        'state_id' : ['nunique'], # mean encoding ?
        'subsector_id' : ['nunique'],
        'purchase_month': ['nunique'],
        'purchase_year': ['nunique'],
        'dayofweek': ['nunique'],
        'weekend': ['sum', 'mean'],
        'month_lag' : ['mean', 'max', 'min', 'std'],
        'month_diff': ['mean', 'max', 'min', 'std'],
    }
    
    agg_transactions = transactions.groupby(['card_id']).agg(agg_func)
    agg_transactions.columns = ['_'.join(col).strip() for col in agg_transactions.columns.values]
    agg_transactions.reset_index(inplace=True)
    
    df = (transactions.groupby('card_id')
          .size()
          .reset_index(name='transactions_count'))
    
    agg_transactions = df.merge(agg_transactions, on='card_id', how='left')
    
    return agg_transactions

agg_historical_transactions = aggregate_transactions(historical_transactions)
agg_new_merchant_transactions = aggregate_transactions(new_merchant_transactions)

Wall time: 3min 58s


In [22]:
agg_historical_transactions.head(2)

,card_id,transactions_count,authorized_flag_sum,authorized_flag_mean,city_id_nunique,category_1_sum,category_1_mean,category_2_1.0_mean,category_2_2.0_mean,category_2_3.0_mean,category_2_4.0_mean,category_2_5.0_mean,category_3_A_mean,category_3_B_mean,category_3_C_mean,installments_sum,installments_mean,installments_max,installments_min,installments_std,merchant_category_id_nunique,merchant_id_nunique,purchase_amount_sum,purchase_amount_mean,purchase_amount_max,purchase_amount_min,purchase_amount_std,purchase_date_min,purchase_date_max,state_id_nunique,subsector_id_nunique,purchase_month_nunique,purchase_year_nunique,dayofweek_nunique,weekend_sum,weekend_mean,month_lag_mean,month_lag_max,month_lag_min,month_lag_std,month_diff_mean,month_diff_max,month_diff_min,month_diff_std
0,C_ID_00007093c1,149,114,0.765101,4,28,0.187919,0.000000,0.0,0.805369,0.0,0.006711,0.0,0.838926,0.161074,192,1.288591,6,1,0.764900,18,29,-76.845041,-0.515739,1.507069,-0.728876,0.298141,2017-02-14 14:00:43,2018-02-27 05:14:57,3,13,12,2,7,25,0.167785,-5.852349,0,-12,3.453114,16.496644,17,16,0.501675
1,C_ID_0001238066,123,120,0.975610,18,2,0.016260,0.772358,0.0,0.000000,0.0,0.162602,0.0,0.715447,0.260163,198,1.609756,10,-1,1.485637,29,65,-72.447201,-0.589002,0.768095,-0.734887,0.190235,2017-09-28 22:25:14,2018-02-27 16:18:59,6,17,6,2,7,52,0.422764,-1.813008,0,-5,1.288980,16.252033,17,16,0.435956


### b3. *merchants*

In [23]:
#merchants = pd.get_dummies(merchants, columns=['category_2'])

### b4. Merge all dataset together

In [24]:
%%time
df_train_test = train_test.merge(agg_historical_transactions,
                                 on='card_id',
                                 how='left')

del agg_historical_transactions
del train_test
gc.collect()

Wall time: 3.93 s


In [25]:
df_train_test.head()

,card_id,feature_3,first_active_month,target,type,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,feature_2_3,elapsed_time,transactions_count,authorized_flag_sum,authorized_flag_mean,city_id_nunique,category_1_sum,category_1_mean,category_2_1.0_mean,category_2_2.0_mean,category_2_3.0_mean,category_2_4.0_mean,category_2_5.0_mean,category_3_A_mean,category_3_B_mean,category_3_C_mean,installments_sum,installments_mean,installments_max,installments_min,installments_std,merchant_category_id_nunique,merchant_id_nunique,purchase_amount_sum,purchase_amount_mean,purchase_amount_max,purchase_amount_min,purchase_amount_std,purchase_date_min,purchase_date_max,state_id_nunique,subsector_id_nunique,purchase_month_nunique,purchase_year_nunique,dayofweek_nunique,weekend_sum,weekend_mean,month_lag_mean,month_lag_max,month_lag_min,month_lag_std,month_diff_mean,month_diff_max,month_diff_min,month_diff_std
0,C_ID_92a2005557,1,2017-06-01,-0.820283,0,0,0,0,0,1,0,1,0,245.0,260,247,0.950000,7,0,0.000000,0.988462,0.0,0.0,0.000000,0.011538,0.984615,0.015385,0.000000,4,0.015385,1,0,0.123314,41,94,-165.968739,-0.638341,2.258395,-0.739395,0.212139,2017-06-27 14:18:08,2018-02-25 09:31:15,3,21,9,2,7,90,0.346154,-3.911538,0,-8,2.397687,16.311538,17,16,0.464015
1,C_ID_3d0044924f,0,2017-01-01,0.392913,0,0,0,0,1,0,1,0,0,396.0,350,339,0.968571,9,31,0.088571,0.911429,0.0,0.0,0.000000,0.000000,0.000000,0.788571,0.205714,543,1.551429,10,-1,1.510777,57,142,-210.006336,-0.600018,4.630299,-0.742400,0.384967,2017-01-06 16:29:42,2018-01-31 22:31:09,3,24,12,2,7,132,0.377143,-5.031429,0,-12,3.804934,17.305714,18,17,0.461369
2,C_ID_d639edf6cd,0,2016-08-01,0.688056,0,0,1,0,0,0,0,1,0,549.0,43,41,0.953488,5,0,0.000000,0.093023,0.0,0.0,0.000000,0.906977,1.000000,0.000000,0.000000,0,0.000000,0,0,0.000000,8,13,-29.167391,-0.678311,-0.145847,-0.730138,0.087380,2017-01-11 08:21:22,2018-02-27 19:08:25,2,7,10,2,7,11,0.255814,-8.604651,0,-13,3.842987,16.232558,17,16,0.427463
3,C_ID_186d6a6901,0,2017-09-01,0.142495,0,0,0,0,1,0,0,0,1,153.0,77,77,1.000000,7,12,0.155844,0.155844,0.0,0.0,0.688312,0.000000,0.000000,0.883117,0.090909,84,1.090909,3,-1,0.588974,25,50,-49.491364,-0.642745,1.445596,-0.740897,0.261624,2017-09-26 16:22:21,2018-02-28 11:44:40,5,13,6,2,7,11,0.142857,-2.831169,0,-5,1.802065,16.285714,17,16,0.454716
4,C_ID_cdbd2c0db2,0,2017-11-01,-0.159749,0,1,0,0,0,0,0,0,1,92.0,133,128,0.962406,6,15,0.112782,0.075188,0.0,0.0,0.804511,0.007519,0.000000,0.947368,0.052632,182,1.368421,12,1,1.896862,26,66,-48.687656,-0.366073,7.193041,-0.746156,1.352094,2017-11-12 00:00:00,2018-02-28 20:40:41,6,17,4,2,7,42,0.315789,-1.285714,0,-3,1.026700,16.278195,17,16,0.449804


In [26]:
%%time
df_train_test.to_pickle('../processed_data/df_train_test.pkl')

Wall time: 1.13 s
